<a href="https://colab.research.google.com/github/LiteraturePro/MOOCCube/blob/master/%E5%AD%A6%E7%94%9F%E9%80%80%E8%AF%BE%E8%A1%8C%E4%B8%BA%E9%A2%84%E6%B5%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 下载数据集
import os
%cd /content
# dowload the file
if not os.path.exists('MCubeData.zip'):
  !gdown --id 1kBWqOG3SZYyeTHCz249sJlVILl0HBnpn \
          -O MCubeData.zip

/content
Downloading...
From: https://drive.google.com/uc?id=1kBWqOG3SZYyeTHCz249sJlVILl0HBnpn
To: /content/MCubeData.zip
100% 594M/594M [00:07<00:00, 81.9MB/s]


In [2]:
# 解压数据集文件
!unzip -o MCubeData.zip -d /content


Archive:  MCubeData.zip
   creating: /content/MCubeData/
  inflating: /content/MCubeData/course_info.json  
  inflating: /content/MCubeData/problem_act_test_new.json  
  inflating: /content/MCubeData/task1_sample.json  
  inflating: /content/MCubeData/user_video_act_train_1.json  
  inflating: /content/MCubeData/user_video_act_train_2.json  
  inflating: /content/MCubeData/user_video_act_train_3.json  
  inflating: /content/MCubeData/user_video_act_train_4.json  
  inflating: /content/MCubeData/user_video_act_val_triple_withId_noLabel_1.json  
  inflating: /content/MCubeData/user_video_act_val_triple_withId_noLabel_2.json  
  inflating: /content/MCubeData/video_info.json  


In [3]:
!sudo apt-get install tree -y
!cd /content/MCubeData
!tree
!cd /content

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  tree
0 upgraded, 1 newly installed, 0 to remove and 37 not upgraded.
Need to get 40.7 kB of archives.
After this operation, 105 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tree amd64 1.7.0-5 [40.7 kB]
Fetched 40.7 kB in 1s (79.6 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package tree.
(Reading database ... 155222 files and directories currently instal

In [4]:
import numpy as np
import pandas as pd
from scipy.stats import pearsonr, kurtosis
import matplotlib.pyplot as plt
# import torch
# from torch import nn,optim
import seaborn as sns
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GroupKFold, KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from lightgbm import LGBMClassifier
import lightgbm as lgb
from itertools import chain
import xgboost as xgb
from xgboost import XGBClassifier
import jieba
import jieba.analyse
import warnings
import os
from tqdm import tqdm
from joblib import Parallel, delayed
import joblib
import json
# from pandarallel import pandarallel
import warnings
warnings.filterwarnings("ignore")

# Initialization
# pandarallel.initialize(progress_bar=False)

In [5]:
# load data
from io import StringIO

def read_json(path):
    file = open(path, "r", encoding="utf-8")
    text = file.read()
    text = text.replace('\'', '\"')
    text = '[' + text.replace('}{','},{') + ']'
    data = pd.read_json(StringIO(text))
    return data
noLabel = read_json('./MCubeData/user_video_act_val_triple_withId_noLabel_2.json')

In [6]:
# load data
from io import StringIO

def read_json(path):
    file = open(path, "r", encoding="utf-8")
    text = file.read()
    text = '[' + text.replace('\n',',')[:-1] + ']'
    data = pd.read_json(StringIO(text))
    return data

In [ ]:
course_info = read_json('./MCubeData/course_info.json')
video_info = read_json('./MCubeData/video_info.json')
train = pd.concat([read_json('./MCubeData/user_video_act_train_1.json'), read_json('./MCubeData/user_video_act_train_2.json')], ignore_index=True)

In [ ]:
course_encode = LabelEncoder().fit(course_info['course_id'].tolist())
video_encode = LabelEncoder().fit(video_info['id'].tolist())

In [ ]:
train['item_id'] = train.index

# create features

In [ ]:
def transform_data(item, istrain=True):
    result = pd.DataFrame.from_records(item['activity'])
    result['item_id'] = item['item_id']
    result.sort_values('local_start_time', inplace=True)
    if istrain:
        result['drop'] = result.course_id.map(dict(zip(item['course_list'], item['label_list'])))
        result['drop'] = result.groupby('course_id')['drop'].transform(lambda x: [0]*(len(x)-1) + [x.iloc[-1]])
    return result

In [ ]:
data = train.apply(transform_data, axis=1)
data = pd.concat(data.to_list(), ignore_index=True)
test = noLabel.apply(lambda x: transform_data(x, False), axis=1)
test = pd.concat(test.to_list(), ignore_index=True)
data = pd.concat([data, test], ignore_index=True)

## couse list doc2vec

In [ ]:
# doc2vec
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

course_documents = [
    TaggedDocument(doc, [i]) for doc, i in zip(
        data.groupby('item_id')['course_id'].apply(lambda x: x.tolist()),
        data.groupby(data['item_id'])['item_id'].apply(lambda x: x.iloc[0]))
]
course_model = Doc2Vec(course_documents,
                       vector_size=16,
                       epochs=7,
                       window=3,
                       min_count=1,
                       workers=16,
                       seed=42,
                       dm=0,
                       hs=1,
                       dbow_words=1)

## video list doc2vec

In [ ]:
video_documents = [
    TaggedDocument(doc, [i]) for doc, i in zip(
        data.groupby('item_id')['video_id'].apply(lambda x: x.tolist()),
        data.groupby(data['item_id'])['item_id'].apply(lambda x: x.iloc[0]))
]
video_model = Doc2Vec(video_documents,
                      vector_size=32,
                      epochs=10,
                      window=3,
                      min_count=1,
                      workers=16,
                      seed=42,
                      dm=0,
                      hs=1,
                      dbow_words=1)

## create_features

In [ ]:
from scipy.spatial.distance import cosine
from joblib import Parallel, delayed


def kfold_mean(df_train, df_test, target, target_mean_list):
    folds = GroupKFold(n_splits=5)

    mean_of_target = df_train[target].mean()

    for fold_, (trn_idx, val_idx) in tqdm(
            enumerate(folds.split(df_train, groups=df_train['item_id']))):
        tr_x = df_train.iloc[trn_idx, :]
        vl_x = df_train.iloc[val_idx, :]

        for col in target_mean_list:
            df_train.loc[vl_x.index, f'{col}_target_enc'] = vl_x[col].map(
                tr_x.groupby(col)[target].mean())

    for col in target_mean_list:
        df_train[f'{col}_target_enc'].fillna(mean_of_target, inplace=True)

        df_test[f'{col}_target_enc'] = df_test[col].map(
            df_train.groupby(col)[f'{col}_target_enc'].mean())

        df_test[f'{col}_target_enc'].fillna(mean_of_target, inplace=True)
    return pd.concat([df_train, df_test], ignore_index=True)


def softmax(x):
    return np.exp(x) / np.sum(np.exp(x))


def attn(u, v):
    return np.sum(softmax(
        1 - (u @ v.T /
             (np.linalg.norm(u, 2) * np.linalg.norm(v, 2, axis=1)))).reshape(
                 (-1, 1)) * v,
                  axis=0).tolist()


def processParallelAttn(df):
    return df.apply(
        lambda y: attn(np.array(y), np.array(df.tolist()))).tolist()

def applyParallel(dfGrouped, func):
    result = []
    tmp = Parallel(n_jobs=-1)(delayed(func)(group) for name, group in tqdm(dfGrouped))
    for a in tmp:
        result.extend(a)
    return result

In [ ]:
from scipy.stats import skew


def create_vec_features(data):

    data[[f'course_vec{i+1}' for i in range(course_model.vector_size)
          ]] = pd.DataFrame(data['course_id'].apply(
              lambda x: course_model.wv.get_vector(x).tolist()).tolist(),
                            index=data.index)
    data[[f'video_vec{i+1}' for i in range(video_model.vector_size)
          ]] = pd.DataFrame(data['video_id'].apply(
              lambda x: video_model.wv.get_vector(x).tolist()).tolist(),
                            index=data.index)

    #     向量表示
    data[[f'item_by_course_vec{i+1}' for i in range(course_model.vector_size)
          ]] = pd.DataFrame(data['item_id'].apply(
              lambda x: course_model.docvecs[x].tolist()).tolist(),
                            index=data.index)
    data[[f'item_by_video_vec{i+1}' for i in range(video_model.vector_size)
          ]] = pd.DataFrame(data['item_id'].apply(
              lambda x: video_model.docvecs[x].tolist()).tolist(),
                            index=data.index)

    data['cosine_of_item_course'] = data.apply(
        lambda x: cosine(course_model.docvecs[x['item_id']],
                         course_model.wv.get_vector(x['course_id'])),
        axis=1)

    print('course attn vec')
    data['course_vec'] = data[[
        f'course_vec{i+1}' for i in range(course_model.vector_size)
    ]].values.tolist()
    data[[
        f'course_vec_attn_mean{i+1}' for i in range(course_model.vector_size)
    ]] = pd.DataFrame(applyParallel(
        data.groupby('item_id')['course_vec'], processParallelAttn),
                      index=data.index)
    del data['course_vec']

    print('video attn vec')
    data['video_vec'] = data[[
        f'video_vec{i+1}' for i in range(video_model.vector_size)
    ]].values.tolist()
    data[[f'video_vec_attn_mean{i+1}' for i in range(video_model.vector_size)
          ]] = pd.DataFrame(applyParallel(
              data.groupby('item_id')['video_vec'], processParallelAttn),
                            index=data.index)
    del data['video_vec']

    for i in tqdm(range(video_model.vector_size)):
        data[f'video_vec_mean{i+1}'] = data.groupby(
            ['item_id', 'course_id'])[f'video_vec{i+1}'].transform('mean')
        data[f'video_vec_max{i+1}'] = data.groupby(
            ['item_id', 'course_id'])[f'video_vec{i+1}'].transform('max')
        data[f'video_vec_min{i+1}'] = data.groupby(
            ['item_id', 'course_id'])[f'video_vec{i+1}'].transform('min')
        data[f'video_vec_std{i+1}'] = data.groupby(
            ['item_id', 'course_id'])[f'video_vec{i+1}'].transform('std')
        data[f'video_vec_skew{i+1}'] = data.groupby(
            ['item_id',
             'course_id'])[f'video_vec{i+1}'].transform(lambda x: x.skew())
    
    return data

In [ ]:
def create_features(data):
    data.sort_values(['item_id', 'local_start_time'], inplace=True)

    #     course/video encode
    print('course/video encode')

    course_dict = course_info.set_index('course_id').to_dict(orient='index')

    data['course_id_count'] = data.groupby(
        data['course_id'])['course_id'].transform('count')
    data['course_id_nunique_video'] = data.groupby(
        data['course_id'])['video_id'].transform('nunique')
    data['course_look_len_mean'] = data.groupby(
        ['item_id', 'course_id'])['video_id'].transform('count').groupby(
            data['course_id']).transform('mean')
    data['course_id_local_watching_time_mean'] = data.groupby(
        data['course_id'])['local_watching_time'].transform('mean')

    data['video_id_count'] = data.groupby(
        data['video_id'])['video_id'].transform('count')
    data['video_id_local_watching_time_mean'] = data.groupby(
        data['video_id'])['local_watching_time'].transform('mean')

    data['course_video'] = data['course_id'].astype(
        str) + data['video_id'].astype(str)
    data = kfold_mean(data[~data['drop'].isna()], data[data['drop'].isna()],
                      'drop', ['course_id', 'video_id', 'course_video'])
    del data['course_video']

    data['video_loc'] = data.apply(
        lambda x: course_dict[x['course_id']]['item'].index(x['video_id']),
        axis=1)
    data['course_len'] = data.apply(
        lambda x: len(course_dict[x['course_id']]['item']), axis=1)
    data['video_loc/course_len'] = data['video_loc'] / data['course_len']

    #     time
    print('time')

    data['local_start_time'] = pd.to_datetime(data['local_start_time'])
    data['local_end_time'] = pd.to_datetime(data['local_end_time'])

    data['year'] = data['local_end_time'].dt.year
    data['month'] = data['local_end_time'].dt.month
    data['day'] = data['local_end_time'].dt.day
    data['dayofweek'] = data['local_end_time'].dt.dayofweek
    data['hour'] = data['local_end_time'].dt.hour

    data = kfold_mean(data[~data['drop'].isna()], data[data['drop'].isna()],
                      'drop', ['year', 'month', 'dayofweek'])

    data['local_watching_time/video_duration'] = data[
        'local_watching_time'] / data['video_duration']
    data['local_watching_time/video_progress_time'] = data[
        'local_watching_time'] / data['video_progress_time']
    data['video_start_time/video_duration'] = data['video_start_time'] / data[
        'video_duration']
    data['video_end_time/video_duration'] = data['video_end_time'] / data[
        'video_duration']
    data['local_watching_time/end-start'] = data['local_watching_time'] / (
        data['video_end_time'] - data['video_start_time'])

    data['watch_time_interval'] = ((data['local_start_time'] - data.groupby(
        'item_id')['local_end_time'].transform(lambda x: x.shift(1))) /
                                   pd.Timedelta(hours=1)).fillna(0)

    #     item
    print('item')

    #     整体统计
    data['item_course_nunique'] = data.groupby(
        'item_id')['course_id'].transform('nunique')

    data['item_video_nunique'] = data.groupby('item_id')['video_id'].transform(
        'nunique')
    data['item_coures_video_nunique'] = data.groupby(
        ['item_id', 'course_id'])['video_id'].transform('nunique')

    data['item_coures_video_watched_ratio'] = data[
        'item_coures_video_nunique'] / data['course_len']

    data['item_watching_count'] = data.groupby(
        'item_id')['watching_count'].transform('sum')
    data['item_coures_video_watching_count'] = data.groupby(
        ['item_id', 'course_id'])['watching_count'].transform('sum')
    data['item_coures_video_watching_count/item_watching_count'] = data[
        'item_coures_video_watching_count'] / data['item_watching_count']

    data['item_watching_time'] = data.groupby(
        'item_id')['local_watching_time'].transform('sum')
    data['item_coures_video_watching_time'] = data.groupby(
        ['item_id', 'course_id'])['local_watching_time'].transform('sum')
    data['item_coures_video_watching_time/item_watching_time'] = data[
        'item_coures_video_watching_time'] / data['item_watching_time']

    data['item_watching_time_interval_mean'] = data.groupby(
        'item_id')['watch_time_interval'].transform('mean')
    data['item_coures_video_watching_time_interval_mean'] = data.groupby(
        ['item_id', 'course_id'])['watch_time_interval'].transform('mean')

    data['item_watching_time_interval_max'] = data.groupby(
        'item_id')['watch_time_interval'].transform('max')
    data['item_coures_video_watching_time_interval_max'] = data.groupby(
        ['item_id', 'course_id'])['watch_time_interval'].transform('max')

    data['item_course_keep_time'] = data.groupby(
        ['item_id', 'course_id'])['local_start_time'].transform(
            lambda x: (x.max() - x.min()) / pd.Timedelta(hours=1))

    data['item_course_final_watch_time_interval'] = (data.groupby(
        ['item_id'])['local_start_time'].transform('max') - data.groupby([
            'item_id', 'course_id'
        ])['local_start_time'].transform('max')) / pd.Timedelta(hours=1)

    data['item_course_next_watch_time_interval'] = data.groupby(
        'item_id')['watch_time_interval'].transform(
            lambda x: x.shift(-1)).fillna(0)


    #     item + course
    print('item + course')
    data['cosine_of_item_course'] = data.apply(
        lambda x: cosine(course_model.docvecs[x['item_id']],
                         course_model.wv.get_vector(x['course_id'])),
        axis=1)

    data['video_loc_mean'] = data.groupby(['item_id', 'course_id'
                                           ])['video_loc'].transform('mean')
    data['video_loc_min'] = data.groupby(['item_id', 'course_id'
                                          ])['video_loc'].transform('min')
    data['video_loc_max'] = data.groupby(['item_id', 'course_id'
                                          ])['video_loc'].transform('max')
    data['video_loc_std'] = data.groupby(['item_id', 'course_id'
                                          ])['video_loc'].transform('std')
    data['video_loc_skew'] = data.groupby(
        ['item_id', 'course_id'])['video_loc'].transform('skew').fillna(0)

    data['video_loc_mean/course_len'] = data['video_loc_mean'] / data[
        'course_len']
    data['video_loc_min/course_len'] = data['video_loc_min'] / data[
        'course_len']
    data['video_loc_max/course_len'] = data['video_loc_max'] / data[
        'course_len']

    data['video_loc_diff'] = data.groupby(
        ['item_id',
         'course_id'])['video_loc'].transform(lambda x: x.diff().fillna(0))
    data['video_loc_diff_mean'] = data.groupby([
        'item_id', 'course_id'
    ])['video_loc'].transform(lambda x: x.diff().mean()).fillna(0)
    data['video_loc_diff_max'] = data.groupby([
        'item_id', 'course_id'
    ])['video_loc'].transform(lambda x: x.diff().max()).fillna(0)
    data['video_loc_diff_min'] = data.groupby([
        'item_id', 'course_id'
    ])['video_loc'].transform(lambda x: x.diff().min()).fillna(0)
    data['video_loc_diff_std'] = data.groupby([
        'item_id', 'course_id'
    ])['video_loc'].transform(lambda x: x.diff().std()).fillna(0)
    data['video_loc_diff_skew'] = data.groupby([
        'item_id', 'course_id'
    ])['video_loc'].transform(lambda x: x.diff().skew()).fillna(0)

    data['video_loc_diff/course_len'] = data['video_loc_diff'] / data[
        'course_len']
    data['video_loc_diff_mean/course_len'] = data[
        'video_loc_diff_mean'] / data['course_len']

    data['video_loc_ddiff'] = data.groupby([
        'item_id', 'course_id'
    ])['video_loc'].transform(lambda x: x.diff().diff().fillna(0))
    data['video_loc_ddiff_mean'] = data.groupby([
        'item_id', 'course_id'
    ])['video_loc'].transform(lambda x: x.diff().diff().mean()).fillna(0)
    data['video_loc_ddiff_max'] = data.groupby([
        'item_id', 'course_id'
    ])['video_loc'].transform(lambda x: x.diff().diff().max()).fillna(0)
    data['video_loc_ddiff_min'] = data.groupby([
        'item_id', 'course_id'
    ])['video_loc'].transform(lambda x: x.diff().diff().min()).fillna(0)
    data['video_loc_ddiff_std'] = data.groupby([
        'item_id', 'course_id'
    ])['video_loc'].transform(lambda x: x.diff().diff().std()).fillna(0)
    data['video_loc_ddiff_skew'] = data.groupby([
        'item_id', 'course_id'
    ])['video_loc'].transform(lambda x: x.diff().diff().skew()).fillna(0)

    data['video_loc_diff_lt0_count'] = data.groupby(
        ['item_id',
         'course_id'])['video_loc_diff'].transform(lambda x: x[x < 0].shape[0])
    data['video_loc_diff_gt1_count'] = data.groupby(
        ['item_id',
         'course_id'])['video_loc_diff'].transform(lambda x: x[x > 1].shape[0])

    data['video_loc_diff_lt0_count/item_coures_video_nunique'] = data['video_loc_diff_lt0_count'] / \
        data['item_coures_video_nunique']
    data['video_loc_diff_gt1_count/item_coures_video_nunique'] = data['video_loc_diff_gt1_count'] / \
        data['item_coures_video_nunique']

    #     item + course_id + time
    print('item + course_id + time')
    for i in [
            'local_watching_time', 'video_progress_time',
            'local_watching_time/video_duration'
    ]:
        data[f'{i}_mean'] = data.groupby(['item_id',
                                          'course_id'])[i].transform('mean')
        data[f'{i}_max'] = data.groupby(['item_id',
                                         'course_id'])[i].transform('max')
        data[f'{i}_min'] = data.groupby(['item_id',
                                         'course_id'])[i].transform('min')
        data[f'{i}_std'] = data.groupby(['item_id',
                                         'course_id'])[i].transform('std')

        data[f'{i}_diff'] = data.groupby(
            ['item_id',
             'course_id'])[i].transform(lambda x: x.diff()).fillna(0)
    
    print('time')
    data['time_end_start'] = data['video_end_time'] - data['video_start_time']
    data['time_end-start/video_duration'] = data[
        'time_end_start'] / data['video_duration']

    print('item + course_id + time')
    data['item_course_video_finished_watching_count'] = data.groupby(['item_id', 'course_id'])[
        'time_end-start/video_duration'].transform(lambda x: x[x >= 1].shape[0])
    data['item_course_video_finished_watching_ratio'] = data['item_course_video_finished_watching_count'] / \
        data['item_coures_video_nunique']
    
    print('target encode stat')
    data['course_id_target_enc_item_mean'] = data.groupby('item_id')['course_id_target_enc'].transform('mean')
    data['course_id_target_enc_item_max'] = data.groupby('item_id')['course_id_target_enc'].transform('max')
    
    data['video_id_target_enc_item_mean'] = data.groupby('item_id')['video_id_target_enc'].transform('mean')
    data['video_id_target_enc_item_max'] = data.groupby('item_id')['video_id_target_enc'].transform('max')
    
    data['video_id_target_enc_item_course_mean'] = data.groupby(['item_id', 'course_id'])['video_id_target_enc'].transform('mean')
    data['video_id_target_enc_item_course_max'] = data.groupby(['item_id', 'course_id'])['video_id_target_enc'].transform('max')
    
   
    
    return data

In [ ]:
data = create_features(data)
data = create_vec_features(data)

course/video encode


5it [00:16,  3.39s/it]


time


5it [00:07,  1.42s/it]


item
item + course
item + course_id + time
time
item + course_id + time
target encode stat
course attn vec


100%|███████████████████████████████████████████████████████████████████████████| 44992/44992 [06:07<00:00, 122.53it/s]


video attn vec


100%|██████████████████████████████████████████████████████████████████████████████████| 32/32 [34:44<00:00, 65.15s/it]


In [ ]:
data.to_csv('./xgb_data.csv', index=None)

## course/video tfidf

In [ ]:
course_video_tfidf = data.groupby('item_id').agg({
    'drop':'sum',
    
    'course_id':lambda x:' '.join(x.tolist()),
    'video_id':lambda x:' '.join(x.tolist()),
})
course_video_tfidf['drop'] = course_video_tfidf['drop'] / course_video_tfidf['course_id'].apply(lambda x: len(set(x)))

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
course_tfidf_vector = TfidfVectorizer(min_df=30).fit(
    course_video_tfidf['course_id'].tolist())
video_tfidf_vector = TfidfVectorizer(min_df=30).fit(
    course_video_tfidf['video_id'].tolist())

In [ ]:
import scipy.sparse as sp
from sklearn.metrics import r2_score


def create_csr_mat_input(course_list, video_list):
    return sp.hstack((course_tfidf_vector.transform(course_list),
                      video_tfidf_vector.transform(video_list)),
                     format='csr')


def kfold_tfidf_model(df_train, df_test, target, model_list):
    folds = KFold(n_splits=5, random_state=42)
    for model in model_list:
        df_test[f'{model.__class__.__name__}_pred'] = 0

    test_x = create_csr_mat_input(df_test['course_id'].tolist(),
                                  df_test['video_id'].tolist())

    for fold_, (trn_idx, val_idx) in tqdm(enumerate(folds.split(df_train))):
        tr_x, tr_y = create_csr_mat_input(
            df_train['course_id'].iloc[trn_idx].tolist(), df_train['video_id'].
            iloc[trn_idx].tolist()), df_train[target].iloc[trn_idx]
        vl_x, vl_y = create_csr_mat_input(
            df_train['course_id'].iloc[val_idx].tolist(), df_train['video_id'].
            iloc[val_idx].tolist()), df_train[target].iloc[val_idx]

        for model in model_list:
            if model.__class__.__name__ == 'LGBMRegressor' or model.__class__.__name__ == 'XGBRegressor':
                model.fit(tr_x,
                          tr_y,
                          eval_set=[(vl_x, vl_y)],
                          early_stopping_rounds=400,
                         verbose=99999)
            else:
                model.fit(tr_x, tr_y)
            y_pred = model.predict(vl_x)
            print(
                f'fold {fold_+1} model: {model.__class__.__name__} r2_score:{r2_score(vl_y, y_pred)}'
            )
            df_train.loc[df_train.iloc[val_idx, :].
                         index, f'{model.__class__.__name__}_pred'] = y_pred

            df_test[f'{model.__class__.__name__}_pred'] += model.predict(
                test_x) / folds.n_splits
    return pd.concat([df_train, df_test])

In [ ]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import PassiveAggressiveRegressor, SGDRegressor, ARDRegression, ElasticNet
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor

estimators = [
    RandomForestRegressor(n_estimators=300,
                          max_depth=20,
                          min_samples_split=20,
                          random_state=42,
                          n_jobs=-1),
    GradientBoostingRegressor(n_estimators=350,
                              learning_rate=0.1,
                              random_state=42,
                              max_features='auto',
                              n_iter_no_change=50),
    PassiveAggressiveRegressor(random_state=42, early_stopping=True),
    SGDRegressor(random_state=42, early_stopping=True),
    LGBMRegressor(n_estimators=9999,
                  learning_rate=0.05,
                  n_jobs=-1,
                  random_state=42,
                  colsample_bytree=0.8,
                  subsample=0.8),
    XGBRegressor(
        n_estimators=9999,
        learning_rate=0.05,
        colsample_bytree=0.8,
        max_depth=10,
        subsample=0.8,
        random_state=42,
        tree_method='gpu_hist',
        predictor='gpu_predictor',
        gpu_id=0
    ),
]

course_video_tfidf = kfold_tfidf_model(
    course_video_tfidf.
    loc[~course_video_tfidf.index.str.startswith('T', na=False), :],
    course_video_tfidf.loc[course_video_tfidf.index.str.
                           startswith('T', na=False), :],
    'drop',estimators
)
course_video_tfidf.to_csv('./course_video_tfidf.csv' , index=None)

In [ ]:
course_video_tfidf = pd.read_csv('./course_video_tfidf.csv')
for model in estimators:
    data[f'{model.__class__.__name__}_pred'] = data['item_id'].map(course_video_tfidf[f'{model.__class__.__name__}_pred'])  

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation
# lda_course = LatentDirichletAllocation(n_jobs=-1,
#                                       random_state=2020,
#                                       n_components=16)
# course_video_tfidf[[
#     f'lda_course{i+1}' for i in range(lda_course.n_components)
# ]] = pd.DataFrame(lda_course.fit_transform(
#     course_tfidf_vector.transform(course_video_tfidf['course_id'].tolist())),
#                   index=course_video_tfidf.index)

# for column_name in [
#     f'lda_course{i+1}' for i in range(lda_course.n_components)
# ]:
#     data[column_name] = data['item_id'].map(course_video_tfidf[column_name])  

In [ ]:
lda_video = LatentDirichletAllocation(n_jobs=-1,
                                      random_state=2020,
                                      n_components=16)
course_video_tfidf[[
    f'lda_video{i+1}' for i in range(lda_video.n_components)
]] = pd.DataFrame(lda_video.fit_transform(
    video_tfidf_vector.transform(course_video_tfidf['video_id'].tolist())),
                  index=course_video_tfidf.index)

for column_name in [f'lda_video{i+1}' for i in range(lda_video.n_components)]:
    data[column_name] = data['item_id'].map(course_video_tfidf[column_name])

In [ ]:
from sklearn.decomposition import NMF
nmf_video = NMF(random_state=2020, n_components=16)
course_video_tfidf[[
    f'nmf_video{i+1}' for i in range(nmf_video.n_components)
]] = pd.DataFrame(nmf_video.fit_transform(
    video_tfidf_vector.transform(course_video_tfidf['video_id'].tolist())),
                  index=course_video_tfidf.index)

for column_name in [f'nmf_video{i+1}' for i in range(nmf_video.n_components)]:
    data[column_name] = data['item_id'].map(course_video_tfidf[column_name])

In [ ]:
from sklearn.decomposition import TruncatedSVD
svd_video = TruncatedSVD(random_state=2020, n_components=16)
course_video_tfidf[[
    f'svd_video{i+1}' for i in range(svd_video.n_components)
]] = pd.DataFrame(svd_video.fit_transform(
    video_tfidf_vector.transform(course_video_tfidf['video_id'].tolist())),
                  index=course_video_tfidf.index)

for column_name in [f'svd_video{i+1}' for i in range(svd_video.n_components)]:
    data[column_name] = data['item_id'].map(course_video_tfidf[column_name])

## drop columns

In [ ]:
drop_columns = [
    'drop', 'item_id', 'local_end_time', 'local_start_time', 'course_year',
    'course_year_month', 'year', 'month', 'day', 'dayofweek', 'hour',
#     'year_target_enc', 'month_target_enc', 'dayofweek_target_enc','course_id_target_enc',
    'video_id', 'course_id'
]

features = [f for f in data.columns if f not in drop_columns]
features = [
    f for f in features if not f.__contains__('video_vec_diff')
    and not f.__contains__('course_year')
]

# fs

In [ ]:
data = data.groupby(['item_id', 'course_id']).tail(1)
data.to_csv('./ddddd.csv', index=None)

In [ ]:
data['course_id'] = course_encode.transform(data['course_id'])
data['video_id'] = video_encode.transform(data['video_id'])
X = data[~data['drop'].isna()]
test = data[data['drop'].isna()]

# run model

In [ ]:
from bayes_opt import BayesianOptimization
from sklearn.model_selection import GroupKFold
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score


def LGB_CV(max_depth,
           learning_rate,
           min_child_weight,
           subsample,
           colsample_bytree,
           reg_alpha,
           reg_lambda,
           X_train=X,
           Y_train=X['drop']):

    folds = GroupKFold(n_splits=5)
    oof = np.zeros(X_train.shape[0])

    for fold_, (trn_idx, val_idx) in enumerate(
            folds.split(X_train, groups=X_train['item_id'].astype(str))):
        #         print("fold n°{}".format(fold_))
        train_x, train_y = X_train[features].iloc[trn_idx], Y_train.iloc[
            trn_idx]
        val_x, val_y = X_train[features].iloc[val_idx], Y_train.iloc[val_idx]
        
        tmp_val_x = X_train.iloc[val_idx, :].sample(
            frac=1, random_state=42).groupby('item_id').head(1)[features]
        tmp_val_y = Y_train.loc[tmp_val_x.index]

        model = XGBClassifier(n_estimators=9999,
                              max_depth=int(max_depth),
                              learning_rate=learning_rate,
                              min_child_weight=int(min_child_weight),
                              subsample=subsample,
                              colsample_bytree=colsample_bytree,
                              reg_alpha=reg_alpha,
                              reg_lambda=reg_lambda,
                              random_state=42,
                              tree_method='gpu_hist',
                              predictor='gpu_predictor',
                              gpu_id=0)

        model.fit(
            train_x,
            train_y,
            eval_set=[(train_x, train_y), (tmp_val_x, tmp_val_y)],
            #             sample_weight=train_x_weight,
            #             eval_sample_weight=[train_x_weight, val_x_weight],
            eval_metric='auc',
            early_stopping_rounds=100,
            verbose=999999)

        oof[val_idx] = model.predict_proba(
            X_train[features].iloc[val_idx])[:, 1]

        del model, trn_idx, val_idx

    return roc_auc_score(Y_train, oof)


LGB_BO = BayesianOptimization(
    LGB_CV, {
        'max_depth': (4, 20),
        'learning_rate': (0.03, 0.08),
        'min_child_weight': (0, 30),
        'subsample': (0.5, 1.0),
        'colsample_bytree': (0.5, 1.0),
        'reg_alpha': (0, 4),
        'reg_lambda': (0, 4),
    })

# LGB_BO.maximize(init_points=2, n_iter=5)
# print(LGB_BO.max)

In [ ]:
from scipy.misc import derivative


def focal_loss_lgb(y_true, y_pred, alpha, gamma):
    a, g = alpha, gamma

    def fl(x, t):
        p = 1 / (1 + np.exp(-x))
        return -(a * t + (1 - a) *
                 (1 - t)) * ((1 - (t * p + (1 - t) *
                                   (1 - p)))**g) * (t * np.log(p) +
                                                    (1 - t) * np.log(1 - p))

    def partial_fl(x):
        return fl(x, y_true)

    grad = derivative(partial_fl, y_pred, n=1, dx=1e-6)
    hess = derivative(partial_fl, y_pred, n=2, dx=1e-6)
    return grad, hess


def focal_loss(x, y):
    return focal_loss_lgb(x, y, 0.25, 1.)


def lgb_f1_score(y_true, y_pred):
    return 'f1', f1_score(y_true, np.round(y_pred)), True


def find_best_t(X):
    best_acc = 0
    best_t = 0.5
    for t in tqdm(range(490, 510)):
        t /= 1000
        #         cur_acc = X.groupby('item_id').parallel_apply(lambda x: accuracy_score(
        #             x['drop'], np.where(x['pred'] >= t, 1, 0))).mean()
        cur_acc = accuracy_score(X['drop'],
                                 np.where(X['pred'] >= t, 1, 0),
                                 sample_weight=1 / X['item_course_nunique'])
        if cur_acc >= best_acc:
            best_t = t
            best_acc = cur_acc
    print(f'best_acc: {best_acc}, best_t:{best_t}')
    return best_t


def lgb_acc(y_true, y_pred, sample_weight):
    return 'acc', accuracy_score(y_true,
                                 y_pred.round(),
                                 sample_weight=sample_weight), True


def kfold_xgb(X, y, test_x):
    test_y = 0
    kf = GroupKFold(n_splits=5)
    X['pred'] = 0
    test_x_weight = 1 / test_x['item_course_nunique']
    for i, (train_index, val_index) in enumerate(
            kf.split(X, y=X['drop'], groups=X['item_id'].astype(str))):
        train_x, train_y = X.iloc[train_index, :][features], y.iloc[
            train_index]
        val_x, val_y = X.iloc[val_index, :][features], y.iloc[val_index]
        tmp_val_x = X.iloc[val_index, :].sample(
            frac=1, random_state=42).groupby('item_id').head(1)[features]
        tmp_val_y = y.loc[tmp_val_x.index]

        train_x_weight = 1 / train_x['item_course_nunique']
        val_x_weight = 1 / val_x['item_course_nunique']

        params = {
            'colsample_bytree': 0.7010984508141815,
            'learning_rate': 0.037811671319995785,
            'max_depth': int(14.356534206010181),
            'min_child_weight': 15.654280256106746,
            'reg_alpha': 2.4870077753307727,
            'reg_lambda': 1.123720654835033,
            'subsample': 0.8312777128028025
        }

        model = XGBClassifier(n_estimators=9999,
                              random_state=42,
                              tree_method='gpu_hist',
                              predictor='gpu_predictor',
                              gpu_id=0,
                              **params)
        model.fit(
            train_x,
            train_y,
            eval_set=[(train_x, train_y), (tmp_val_x, tmp_val_y)],
            #             sample_weight=train_x_weight,
            #             eval_sample_weight=[train_x_weight, val_x_weight],
            eval_metric='auc',
            early_stopping_rounds=400,
            verbose=100)
        print(
            f'{i+1} fold accuracy_score is {accuracy_score(val_y, model.predict(val_x), sample_weight=val_x_weight)}'
        )
        #

        #         retrain
        best_iteration_ = model.best_iteration
#         retrain_x = test_x
        retrain_x = pd.concat([test_x, val_x])
        #         first round
        retrain_idx = model.predict_proba(retrain_x)[:, 1]
        retrain_idx = (retrain_idx >= 0.95) | (retrain_idx <= 0.05)
        copy_test_x = retrain_x.copy().iloc[retrain_idx, :]
        copy_test_y = pd.Series(model.predict(copy_test_x))
        model.set_params(n_estimators=best_iteration_ + 10, verbose=300)
        model.fit(
            pd.concat([train_x, copy_test_x]),
            pd.concat([train_y, copy_test_y]),
            #                   sample_weight=pd.concat([train_x_weight, test_x_weight])
        )
        #         second round
        retrain_idx = model.predict_proba(retrain_x)[:, 1]
        retrain_idx = (retrain_idx >= 0.95) | (retrain_idx <= 0.05)
        copy_test_x = retrain_x.copy().iloc[retrain_idx, :]
        copy_test_y = pd.Series(model.predict(copy_test_x))
        model.set_params(n_estimators=best_iteration_ + 10, verbose=300)
        model.fit(
            pd.concat([train_x, copy_test_x]),
            pd.concat([train_y, copy_test_y]),
            #                   sample_weight=pd.concat([train_x_weight, test_x_weight])
        )    
        
        print(
            f'{i+1} fold retrain accuracy_score is {accuracy_score(val_y, model.predict(val_x), sample_weight=val_x_weight)}'
        )

        X.loc[val_x.index, 'pred'] = model.predict_proba(val_x)[:, 1]
        test_y += model.predict_proba(test_x) / kf.n_splits

    total_acc = X.groupby('item_id').apply(
        lambda x: accuracy_score(x['drop'], x['pred'].round())).mean()
    print(f'total accuracy_score is {total_acc}')

    #         post oper
    #     t = find_best_t(X)
    t = 0.5

    xgb.plot_importance(model, max_num_features=30, height=0.5)
    plt.show()
    return np.where(test_y[:, 1] >= t, 1, 0), test_y[:, 1]

In [ ]:
X.fillna(0, inplace=True)
y_pred, y_pred_prob = kfold_xgb(X, X['drop'], test[features].fillna(0))
# total accuracy_score is 0.8657915776254426 online:0.867983668515019

# submit

In [ ]:
# test['drop'] = y_pred.argmax(axis=1)
test['drop'] = y_pred
test['drop_prob'] = y_pred_prob
test['course_id'] = course_encode.inverse_transform(test['course_id'])

In [ ]:
test[['item_id', 'course_id', 'drop_prob']].to_csv('./test_drop_prob_xgb.csv', index=None)

In [ ]:
noLabel['label_list'] = noLabel.apply(lambda x: [
    test.loc[(test['item_id'] == x['item_id']) &
             (test['course_id'] == c), 'drop'].iloc[0]
    for c in x['course_list']
],
    axis=1)

In [ ]:
with open('./submit/submit_xgb.json',"w+") as f:    #设置文件对象
    f.write(noLabel[['label_list', 'item_id']].to_json(orient='records')[1:-1].replace(',{', '\n{'))

In [ ]:
X[['item_id', 'course_id', 'pred']].to_csv('./train_pred.csv', index=None)